# Modelos de Lenguaje de OpenAI

A mitad de febrero, [OpenAI publicó un modelo de lenguaje](https://blog.openai.com/better-language-models/) capaz de generar lenguaje natural de formar coherente. Este modelo es generalista y, a pesar de ello, es capaz de rivalizar con los mejores sistemas específicos en tareas como comprensión automática de lenguaje natural, traducción automática, búsqueda de respuestas y resumen automático.

Este modelo, llamado GPT-2, es el resultado de haber entrenado con 8 millones de páginas web (40 GB) con 1 500 millones de parámetros con un único objetivo: predecir cuál es la siguiente palabra.

Sin embargo, OpenAI no ha publicado el modelo para evitar que alguien con malas intenciones pueda hacer un uso dañino de esta tecnología. Sí que han publicado una versión simplificada y más pequeña, y el paper ["Language Models are Unsupervised Multitask Learners"](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), en el que explican todo el proceso.

Con ganas y GPUs suficientes (+ tiempo y dinero), se puede replicar el proceso. Otras lecturas interesantes, sobre el tema: 

- [OpenAI's new Multitalented AI Writes, Translates, and Slanders](https://www.theverge.com/2019/2/14/18224704/ai-machine-learning-language-models-read-write-openai-gpt2)
- [Some thoughts on zero-day threats in AI, and OpenAI's GP2](https://www.fast.ai/2019/02/15/openai-gp2/)


Este código de ejemplo está inspirado en [un tweet de Thomas Wolf](https://twitter.com/Thom_Wolf/status/1097465312579072000), de [Hugging Face](https://huggingface.co/).

In [2]:
# asegúrate de tener instaladas las siguientes dependencias
!pip3 install numpy torchvision_nightly
!pip3 install torch_nightly -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
!pip3 install pytorch_pretrained_bert

  Using cached https://files.pythonhosted.org/packages/35/d5/4f8410ac303e690144f0a0603c4b8fd3b986feb2749c435f7cdbb288f17e/numpy-1.16.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c8/d8/6bb4584b20459cc057f1b922fe7e4757d005c812d2cf73d2903a86a8afd6/torchvision_nightly-0.2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/c2/f84b1e57416755e967236468dcfb0fad7fd911f707185efc4ba8834a1a94/Pillow-6.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6c/4b/c38b5144cf167c4f52288517436ccafefe9dc01b8d1c190e18a6b154cd4a/tqdm-4.31.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 100.6MB 11kB/s 
  Using cached https://files.pythonhosted.org/packages/5d/3c/d5fa084dd3a82ffc645aba78c417e6072ff48552e3301b1fa3bd711e03d4/pyto

In [3]:
import torch 
from torch.nn import functional as F 
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 176/176 [00:00<00:00, 68307.35B/s]


A continuación, definimos una función para:

1. tokenizar el texto de entrada y codificarlo como un vector con los pesos obtenidos por el modelo GPT2
2. predecir la siguiente palabra más frecuente
3. decodificar el vector como una secuencia de tokens

In [4]:
def generate(text, length=50):
    """Generate automatic Natural Language from the input text"""
    vec_text = tokenizer.encode(text)
    my_input, past = torch.tensor([vec_text]), None
    
    for _ in range(length):
        logits, past = model(my_input, past=past)
        my_input = torch.multinomial(F.softmax(logits[:, -1]), 1)
        vec_text.append(my_input.item())
    
    return tokenizer.decode(vec_text)

In [5]:
# defino un texto de entrada
text = "After President Trump was pressured by his son-in-law and daughter"

# y generamos automáticamente las secuencias más probables
print(generate(text, 75))

/home/eduardo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


After President Trump was pressured by his son-in-law and daughter, Ivanka Trump, to yank 100,000 jobs from the United States from North Korea, many and many full time, new reports emerged that reflect Trump's untruths about the Iron Dome.Internal Cruz campaign counsel Brian Fallon claims only "90 per cent of generals heard of the principles Trump adopted" and Earl of Thurmond. He suggests even know people under the
